In [3]:
import sys
from aerospike import Client
from aerospike import predicates as pred
from set_proxy import SetProxy
from open_ai_gateway import recommend
from pprint import pprint

In [4]:
def clean_and_aggregate(accum_list, callback_record):
    record_body = callback_record[2]
    record_body["itemId"] = int(record_body["itemId"])
    accum_list.append(record_body)

In [5]:
cluster_addrs = {
        "hosts": [("localhost", 3000)]
    }
client = Client(cluster_addrs).connect( )

In [6]:
proxy = SetProxy(client, "inventory", "catalog")

In [7]:
cleaned_results = [ ]

In [8]:
dresses_query = proxy.query( )

In [9]:
dresses_query.where(pred.equals("category", "dress"))

In [10]:
dresses_query.select("itemId", "subcategory", "decade", "pattern", "color",
                "size", "material")

In [11]:
dresses_query.foreach(lambda curr_record: clean_and_aggregate(
            cleaned_results, curr_record))

In [12]:
cleaned_results

[{'itemId': 3482,
  'subcategory': 'mumu',
  'decade': '1950',
  'pattern': 'geometric',
  'color': ['bronze', 'graphite', 'cerulean'],
  'size': '18',
  'material': 'silk'},
 {'itemId': 4297,
  'subcategory': 'bodycon',
  'decade': '1950',
  'pattern': 'solid',
  'color': ['mulberry'],
  'size': '14',
  'material': 'linen'},
 {'itemId': 5831,
  'subcategory': 'trapeze',
  'decade': '1960',
  'pattern': 'paisley',
  'color': ['amethyst', 'eggplant', 'butterscotch'],
  'size': '12',
  'material': 'chiffon'},
 {'itemId': 4247,
  'subcategory': 'halter',
  'decade': '1990',
  'pattern': 'solid',
  'color': ['rouge'],
  'size': '6',
  'material': 'linen'},
 {'itemId': 5036,
  'subcategory': 'sundress',
  'decade': '1940',
  'pattern': 'floral',
  'color': ['eggplant', 'lime', 'magenta', 'olive green'],
  'size': 'S',
  'material': 'flannel'},
 {'itemId': 5690,
  'subcategory': 'empire waist',
  'decade': '1990',
  'pattern': 'plaid',
  'color': ['rouge', 'onyx', 'ivory', 'periwinkle'],
  '

In [13]:
recommend(2900, cleaned_results)

Making RAR (retrieval-augmented recommendation) call to OpenAI...
OpenAPI call complete. Round trip time = 5.877 sec.


{'top3': [{'itemId': 5745,
   'score': 0.8,
   'why': 'Bodycon silhouette matches exactly, 1990s era is close to 1980s, solid pattern, color is a jewel tone, and size is within acceptable range.'},
  {'itemId': 5140,
   'score': 0.75,
   'why': 'Minidress silhouette is similar to bodycon, 1980s era matches, solid pattern, color is a jewel tone, but size is larger.'},
  {'itemId': 5615,
   'score': 0.7,
   'why': 'Empire waist is less similar, but 1980s era matches, solid pattern, color is a jewel tone, and material is luxurious.'}],
 'notes': "The top match is a bodycon dress from the 1990s, which closely aligns with the exemplar's silhouette and era, though the color is different. The second choice is a minidress from the 1980s, which shares the era and pattern but differs in size. The third option is an empire waist dress from the 1980s, which matches the era and pattern but has a less similar silhouette. All selections prioritize silhouette and era, with some trade-offs in color and